## User Guide on HTMLCleanser and HTMLSplitter
:: This guide introduces way to use the HTMLCleanser and HTMLSplitter.

- Its purpose is on introducing the HTMLCleanser to preprocess the html (e.g. cleanse out invalid html tags and attributes).
- Its purpose is on introducing the HTMLSplitter to split the documents before indexing the documents using IR model to perform RAG.
- It supposes that your document is of the format `html`.



:: Main modules: [cleanser.py, splitter.py]

- The system pre-processes files in the html format. If your document is in another format, please convert it to an HTML file beforehand.

- The `HTMLCleanser` in cleanser.py removes invalid tags and attributes within the BeautifulSoup object.
- The `HTMLSplitter` in splitter.py divides the file into several documents without altering the HTML contents.
    - When developing an AI model or utilizing the LLM, the existence of token_max can make it challenging to input the entire document into the model.
    - Consequently, you may have been splitting the documents to avoid exceeding the token_max, using strategies such as doc_stride, etc.
    - If you simply truncate the document based on the token length, the content may be chopped off, resulting in a loss of context.
    - The `HTMLSplitter` takes into account the token_max when splitting the file, ensuring that the HTML file is divided without compromising context.

- [as-is] The contents (e.g., tables) inside the document were truncated without considering the context due to the maximum token limit imposed by the AI model.

- [to-be] The contents (e.g., tables) inside the document will be split without losing context and structure, with a guarantee that the splitted chunks do not exceed the maximum tokens.

    Firstly, split the target file into a set of documents before indexing, while considering the maximum token limit accepted by the model.

    Secondly, insert the splitted documents into the model as usual.

- For detailed usage instructions of the modules, kindly consult the `guide.ipynb` file located within the html directory.

[TODO]
- Add more funtionality in HTMLSplitter._split_chunk 
- Think of how to handle the context window 

```
- Writer: Eungi Cho
- Last update: 23.11.21
```

In [1]:
from bs4 import BeautifulSoup
from _html.cleanser import HTMLCleanser

# suppose you have the html file as below:
txt = """
<table>
    <thead>
        <tr>
            <th>Header 1</th>
            <th>Header 2</th>
            <th>Header 3</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Data 1</td>
            <td>Data 2</td>
            <td>Data 3</td>
        </tr>
        <tr>
            <td>Data 4</td>
            <td>Data 5</td>
            <td>Data 6</td>
        </tr>
    </tbody>
</table>
"""
# get soup object by specifying the parser
soup = BeautifulSoup(txt, "lxml")

# usage of cleanser: it will print out default valid_tags and valid_attributes
# to leave in the text when it is initialized.
cleanser = HTMLCleanser()
# add or remove any tag as your pleases.
cleanser.add_valid_tags(["p", "img", "a", "span", "title"])
cleanser.remove_valid_tags(["span"])
# add or remove any attributes as your pleases.
cleanser.add_valid_attrs(["href", "src", "alt", "font"])
cleanser.remove_valid_attrs(["font"])

# cleanse your html first.
soup = cleanser.cleanse_html(soup)

2023-11-21 17:12:25.196 | INFO | _html.cleanser:__init__:40 - No valid_tags or valid_attrs provided.
                    Initialize the HTMLCleanser with default ones:
                    - valid_tags: ['table', 'tr', 'td', 'th']
                    - valid_attrs: ['rowspan', 'colspan']
2023-11-21 17:12:25.197 | INFO | _html.cleanser:add_valid_tags:67 - valid_tags: ['a', 'p', 'title', 'img', 'tr', 'td', 'table', 'span', 'th']
2023-11-21 17:12:25.198 | INFO | _html.cleanser:remove_valid_tags:77 - valid_tags: ['a', 'p', 'title', 'img', 'tr', 'td', 'table', 'th']
2023-11-21 17:12:25.199 | INFO | _html.cleanser:add_valid_attrs:73 - valid_attrs: ['src', 'alt', 'colspan', 'href', 'font', 'rowspan']
2023-11-21 17:12:25.200 | INFO | _html.cleanser:remove_valid_attrs:81 - valid_attrs: ['src', 'alt', 'colspan', 'href', 'rowspan']


In [2]:
from _html.splitter import HTMLSplitter

splitter = HTMLSplitter(soup=soup, length_func=len, token_max=200)
chunks = splitter.get_chunks()
chunks = splitter.split_chunks(chunks)
documents = splitter.make_documents(chunks)

In [4]:
print(f"Number of documents: {len(documents)}")

for doc in documents:
    print(f"----------- Page {doc.page} | Tokens {len(doc.page_content)} -----------")
    print(doc.page_content)
    print(f"----------------------------------------")
    print("\n\n")

Number of documents: 2
----------- Page 0 | Tokens 199 -----------
<table>
 <tr>
  <th>
   Header 1
  </th>
  <th>
   Header 2
  </th>
  <th>
   Header 3
  </th>
 </tr>
 <tr>
  <td>
   Data 1
  </td>
  <td>
   Data 2
  </td>
  <td>
   Data 3
  </td>
 </tr>
</table>

----------------------------------------



----------- Page 1 | Tokens 199 -----------
<table>
 <tr>
  <th>
   Header 1
  </th>
  <th>
   Header 2
  </th>
  <th>
   Header 3
  </th>
 </tr>
 <tr>
  <td>
   Data 4
  </td>
  <td>
   Data 5
  </td>
  <td>
   Data 6
  </td>
 </tr>
</table>

----------------------------------------



